# Gulf of Corinth evolution over the last 130 kyrs

To simulate the surface evolution of the Corinth rift over the past 130 kyrs, our input file integrates non-linear hillslope diffusion, and river incision by means of the stream power law (_default fluvial incision_ -- **detachment-limited**), allowing for balanced sediment transport and deposition under varying tectonic and climatic forcings. 

***

<img src="images/model.jpg" alt="geometry" width="900" height="500"/>

***

This example is based on the following paper:

**Pechlivanidou S., P. Cowie, G. Duclaux, N. Nixon, R. Gawthorpe, T. Salles: Tipping the balance: Shifts in sediment production in an active rift setting, Geology, [doi:10.1130/G45589.1](https://doi.org/10.1130/G45589.1), 2019.**

***


Erosion rate $\dot{\epsilon}$ depends on drainage area $A$, net precipitation $P$ and local slope $S$ and takes the form:

$\dot{\epsilon}=\kappa_{d} P^l (PA)^m S^n$

$\kappa_{d}$ is a dimensional coefficient describing the erodibility of the channel bed as a function of rock strength, bed roughness and climate, $l$, $m$ and $n$ are dimensionless positive constants. Default formulation assumes $l=0$, $m=0.5$ and $n=1$. The precipitation exponent $l$ allows for representation of climate-dependent chemical weathering of river bed across non-uniform rainfall.

In our simulation, we assume that:

+ $l=0$, $m=0.5$ and $n=1$
+ $\kappa_{d} = 2.e^{-6}$

> This value of $\kappa_{d}$ will only be used for already mobilised sediment as we also impose underlying erodibility layers in this example. 




In [ ]:
import glob
import lavavu

import cmocean as cmo
from matplotlib import cm


from scripts import viewBadlands as visu
from scripts import catchmentErosion as eroCatch

from badlands.model import Model as badlandsModel

%config InlineBackend.figure_format = 'svg'
%matplotlib inline

## Starting badlands

First, we initialise the model and set the path to the **XmL** input file:

+ **input.xml**

> You can edit the **XmL** configuration file directly in the _Jupyter environment_ by clicking on it in the `tree` and changing in the **url** the **view** to **edit**.

In [ ]:
# Initialise model
model = badlandsModel()

# Define the XmL input file
model.load_xml('input.xml')

## Running badlands

We can run the model for a given period. The end time in the **XmL** input file is set to _130 ky_ but you might want to run the model for a coupled of iterations and check the output before running the model for the entire simulation time. This is done by putting the time in the **run_to_time** function. 

Here we go for the full time directly...

In [ ]:
model.run_to_time(130000)

## Visualise outputs

In [ ]:
folder = 'output/'
stepCounter = len(glob.glob1(folder+"/xmf/","tin.time*"))-1
visu.viewTime(folder, steps=stepCounter, it=1, scaleZ=10, maxZ=2000, maxED=350)

## Loading output

We first load the simulation last output file (timestep=13) to pick a specific catchment.

In [ ]:
catchment = eroCatch.catchmentErosion(folder='output',timestep=13)
catchment.regridTINdataSet()

To extract the information from a particular catchment we only need to get a point belonging to this catchment. Here we chose the following point coordinates:

+ `pt=[637936.,4210856.]`

Let's see its position within the region...

In [ ]:
catchment.plotdataSet(title='Elevation', data=catchment.z, color=cmo.cm.delta,  
                      crange=[-2000,2000], pt=[637936.,4210856.])

> We can zoom in to ensure that this is at the right spot... beware that the function will not work if the point is defined underwater!

We specify the extent of the zoom region using:

+ `erange=[x1,x2,y1,y2]`

In [ ]:
catchment.plotdataSet(title='Elevation', data=catchment.z, color=cmo.cm.delta,  
                      crange=[-2000,2000], pt=[637936.,4210856.],
                      erange=[618000,649000,4190000,4230000])

### Performing erosion analyse on the chosen catchment

Now that we are sure about our catchment position we extract from the TIN the relevant points

In [ ]:
catch = eroCatch.catchmentErosion(folder='output', timestep=13, pointXY=[637936.,4210856.])

In [ ]:
catch.regridTINdataSet()

The alpha parameter is used to perform the delineation of the catchment and define a convex hull.

In [ ]:
catch.extractRegion(alpha=0.001)

We can visualise the catchment, the points which have been taken to perform the analyse as well as the contour of the defined area. 

In [ ]:
catch.plotCatchment()

Based on the extracted points from the figure above we can compute the **cumulative erosion** over the duration of the simulation (130 ky) as well as the **eroded sediment rate**. 

In [ ]:
catch.getErodedVolume(time=130000.)

Obviously all the data outputed by badlands can be further process to look at other things depending of what you think will be useful...

Here as an example, we can look at the **distribution of erosion rate versus elevation across the entire catchment**:

In [ ]:
catch.plotEroElev(time=130000.)

### Morphometrics

<div class="alert alert-block alert-info">We will now extract the value of deposited volume for a specific area and plot other metrics available to analyse the surface morphology.</div>

We can plot a series of different parameters from our landscape:

+ elevation: `dataTIN.z`
+ slope: `dataTIN.slp`
+ erosion/deposition: `dataTIN.dz`
+ aspect: `dataTIN.aspect`
+ horizontal curvature: `dataTIN.hcurv`
+ vertical curvature: `dataTIN.vcurv`

In [ ]:
catchment.plotdataSet(title='Slope', data=catchment.slp, color=cmo.cm.tempo,crange=[0.,1.])
catchment.plotdataSet(title='Erosion/Deposition [m]', data=catchment.dz, color=cmo.cm.balance,crange=[-300.,300.])
catchment.plotdataSet(title='Aspect', data=catchment.aspect, color=cmo.cm.haline,crange=[0.,2.],ctr='w')
catchment.plotdataSet(title='Horizontal curvature', data=catchment.hcurv, color=cmo.cm.balance,
                      crange=[-0.001,0.001])
catchment.plotdataSet(title='Vertical curvature', data=catchment.vcurv, color=cmo.cm.balance,
                      crange=[-0.0012,0.0012])

### Finding deposited sediment volume

As shown in the previous notebook on the erosion rate, we have access to the deposited volume over time and can perform from the notebook a calculation of the accumulated volume in the entire region or in a specific area. This can be done in this way:

In [ ]:
catchment.plotdataSet(title='Erosion/Deposition [m]', data=catchment.dz, color=cmo.cm.amp,  
                      crange=[0,500], erange=[570000,665000,4200000,4260000],
                      depctr=(50,150,300,500),size=(10,10))

In [ ]:
catchment.getDepositedVolume(time=130000.,erange=[570000,665000,4200000,4260000])